## Predicting the time taken to process a green card application in the US

The goal of this project was to predict the time it takes to process a permanent work visa application (green card) in the US. This is useful for individuals applying for a job to plan their job search process, and for companies to better tailor their hiring process. 

** Note : I have not figured out how to get plotly plots to display on github. To view the entire projects and plots please click here (link to nbviewer):** [(here)](https://goo.gl/gc0Qfe)

I downloaded the data from the US Dept. of Labor Forces [(here)](https://www.foreignlaborcert.doleta.gov/performancedata.cfm)

I then proceeded to:

1. Clean the data
2. Explore and Visualize the data to gain some insights 
3. Run some statistical tests to see if certain features had an impact on the processing time
4. Use Linear Regression to predict the number of days to process a visa
5. Use Logistic Regression to predict whether the processing time fell within a certain rage, and compare the performance to Linear Regression

I am currently working on:

1. Implementing other Machine Learning Algorithms like Random Forests
2. Trying to find better features that might improve the predictions
3. Adding more data
4. Developing a web app, so that a potential visa applicant can key in his information and get a predicted wait time



In [3]:
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
#import us
#import os
#from iso3166 import countries

perm_data = pd.read_pickle('perm')


Let’s have a look at the data from 2015.

There were 125 columns, most of which were not relevant to this analysis. Some examples of these less relevant columns were the information of the attorney who filled out the visa application, or the name of the newspaper with the job posting. 

I kept 13 columns I felt were most relevant, and can include additional ones if needed based on the results of the analysis.


The columns are as follows:


1. CASE_NUMBER: Unique Identification number for each applicant. (e.g. A-14220-96665; ‘A’ – Atlanta Processing Center; ‘14’ - last 2 digits of year application was created; 220 - Julian date application was created)
2. DECISION_DATE: Date decision was reached on status of application
3. CASE_STATUS: ‘Certified’ and ‘Certified-Expired’ : Application was accepted. ‘Denied’ : Application denied. ‘Withdrawn’ : Application withdrawn by applicant
4. CASE_RECEIVED_DATE: Date application was received by Processing Center
5. EMPLOYER_NAME: Name of employer
6. PW_JOB_TITLE_9089: Type of job (Falls into categories like Software Developer, Pharmacists, etc)
7. PW_LEVEL_9089: Level of experience of applicant. Level 1: Entry level; Level 2: Qualified; Level 3: Experienced; Level 4: Fully Competent
8. WAGE_OFFER_FROM_9089: Wage offer
9. WAGE_OFFER_UNIT_OF_PAY_9089: Hourly / Weekly / Bi-weekly/Monthly/ Yearly
10. JOB_INFO_WORK_CITY: City of Employment
11. JOB_INFO_WORK_STATE: State of Employment
12. COUNTRY_OF_CITIZENSHIP: Citizenship of Applicant
13. CLASS_OF_ADMISSION: Immigration visa the worker currently holds


In [5]:
perm_data_15 = perm_data[pd.DatetimeIndex(perm_data['DECISION_DATE']).year == 2015]
perm_data_15.head()

CASE_NUMBER DECISION_DATE        CASE_STATUS CASE_RECEIVED_DATE  \
15 0  A-14220-96665    2015-02-03  Certified-Expired         2014-09-10   
   1  A-14220-96720    2015-01-12  Certified-Expired         2014-08-08   
   3  A-14206-92509    2015-03-10  Certified-Expired         2014-10-19   
   4  A-14202-90786    2015-01-07  Certified-Expired         2014-08-25   
   6  A-14202-90920    2015-01-26  Certified-Expired         2014-07-25   

                            EMPLOYER_NAME  \
15 0               UNION PACIFIC RAILROAD   
   1                   VST CONSULTING INC   
   3                    INTEL CORPORATION   
   4           NET ESOLUTIONS CORPORATION   
   6  BURGER REHABILITATION SYSTEMS, INC.   

                           PW_JOB_TITLE_9089 PW_LEVEL_9089  \
15 0       Software Developers, Applications     Level III   
   1       Software Developers, Applications     Level III   
   3  Electronics Engineers, Except Computer           NaN   
   4       Software Developers, Applications      Level II   
   6                  Occupational Therapist      Level II   

     WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089 JOB_INFO_WORK_CITY  \
15 0            93,000.00                        Year              Omaha   
   1            90,459.00                        Year             Iselin   
   3            80,617.00                        Year             Folsom   
   4            94,016.00                        Year             McLean   
   6           104,000.00                        Year        Porterville   

     JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP CLASS_OF_ADMISSION  
15 0            NEBRASKA                  INDIA               H-1B  
   1          NEW JERSEY                  INDIA               H-1B  
   3          CALIFORNIA             BANGLADESH               H-1B  
   4            VIRGINIA                  INDIA               H-1B  
   6          CALIFORNIA            PHILIPPINES               H-1B

**Cleaning the data:**

There were a total of 98453 applications processed in 2015. 

First, I drop any rows with null values. 

That leaves me with 85556 Applications, still a large dataset. 

*Note about null values: Most of the null values come from incomplete Wage Levels and Class of Admission. I thought it might be that individuals who did not hold an immigrant visa in the US did not fill out the ‘Class of Admission’ section. However, there were many other individuals who filled out ‘Not in USA’. There were also individuals who were in the US on a H1-B who had not filled out this section. Since this was unclear and I wanted to use Wage Levels and Class of Admission as a feature, I dropped all null values.*


In [6]:
print(str(len(perm_data_15)) + ' Total Applications in 2015' + '\n')
print(perm_data_15.isnull().sum())

print('\n' + str(len(perm_data_15.dropna(how='any'))) + ' remaining applications after nulls dropped')
perm_data_15 = perm_data_15.dropna(how='any')

98453 Total Applications in 2015

CASE_NUMBER                       0
DECISION_DATE                     0
CASE_STATUS                       0
CASE_RECEIVED_DATE                1
EMPLOYER_NAME                     4
PW_JOB_TITLE_9089                37
PW_LEVEL_9089                  6041
WAGE_OFFER_FROM_9089             34
WAGE_OFFER_UNIT_OF_PAY_9089    1021
JOB_INFO_WORK_CITY               18
JOB_INFO_WORK_STATE              20
COUNTRY_OF_CITIZENSHIP           15
CLASS_OF_ADMISSION             6702
dtype: int64

85556 remaining applications after nulls dropped


Second, I drop all cases where the application was ‘Withdrawn’ because these were incompletely processed and it doesn't make sense to take them into account.

So all in all I have 81740 Applications.

Out of them, 77253 applications were accepted and 4487 applications were denied.

Now that the data is cleaner, I add a 14th column, ‘TIME_TAKEN’, that has the number of days it took to process the application.

In [7]:
perm_data_15 = perm_data_15[perm_data_15['CASE_STATUS'].str.startswith('W') == False]
print(str(len(perm_data_15)) + ' Total Applications Processesd' + '\n')
print(str(len(perm_data_15[perm_data_15['CASE_STATUS'].str.startswith("C")])) + " applications were accepted and " + str(len(perm_data_15[perm_data_15['CASE_STATUS'].str.startswith("D")]))+ " applications were denied.")
td = (perm_data_15['DECISION_DATE'] - perm_data_15['CASE_RECEIVED_DATE'])
perm_data_15["TIME_TAKEN"] = (td / np.timedelta64(1, 'D')).astype(int)

81740 Total Applications Processesd

77253 applications were accepted and 4487 applications were denied.


Next, I wanted to check if there were any duplicates in the data. I found 37 duplicates. While looking through the duplicates, I realised there might be more duplicates, especially if 2 applications have the same employer name or job title, but the text doesnt match (the spaces, commas or upper / lower case is different). 

I decided to check for duplicates in the columns that had numbers or very uniform text (d1). I found 1026 duplicates! Just as a sanity check, I also checked for duplicates just using the Case Number - because each applicant gets a unique case number. The no of duplicates increased to 1064 (d2). 

I was confused as to why the number of duplicates increased. I compared the two sets of duplicates (d1 and d2), and found that the Decision Dates and Case Status differed even though the 2 cases would have the same case number. The earlier decision date would have a Status of "Denied" and the later decision date would have a "Certified" status. I looked into this online and found out that sometimes the DOL sends out erroneous decisions, and these decisions can be appealed. The DOL then updates the Case Status once it makes its new decision - So both of these applications are actually just from one applicant and the one with the earlier Decision Date should be removed since that reflects an incorrect processing time.


In [8]:
print(len(perm_data_15))
print(str(len(perm_data_15[perm_data_15.duplicated()]))+' absolute duplicates in the data')
d1 = perm_data_15[perm_data_15.duplicated(subset=['DECISION_DATE', 'CASE_NUMBER', 'CASE_RECEIVED_DATE', 'PW_LEVEL_9089'])]
d2 = perm_data_15[perm_data_15.duplicated(subset=['CASE_NUMBER'])]
print(str(len(d1))+' duplicates if I dont include columns that have confusing text input')
#print(d2[~d2['CASE_NUMBER'].isin(d1['CASE_NUMBER'])])
print(str(len(d2)) + ' duplicates of case number')

perm_data_15 = perm_data_15.drop_duplicates(subset=['CASE_NUMBER'], keep = 'last')
print(len(perm_data_15))


81740
37 absolute duplicates in the data
1026 duplicates if I dont include columns that have confusing text input
1064 duplicates of case number
80676


While looking through the duplicates data, I also found some applications with different case numbers, but they had exactly the same details in all the other columns. I was worried that applicants might be filing multiple applications just to increase their chances of getting a visa. 

I decided to investigate this further, and accessed additional details about these application from this website: http://dolstats.com/. I found that there were differences in these applications: for example, they had differing levels of education, and where they were educated were different,  so I concluded that they were different persons. Maybe the company got a lot of applications at that time, or filed multiple applications around the same time.



In [9]:
d3 = perm_data_15[perm_data_15.duplicated(subset=['DECISION_DATE', 'CASE_RECEIVED_DATE', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'JOB_INFO_WORK_CITY', 'COUNTRY_OF_CITIZENSHIP', 'EMPLOYER_NAME' , 'PW_JOB_TITLE_9089', 'PW_LEVEL_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_UNIT_OF_PAY_9089'], keep=False)]
d3.tail()
#d4 = perm_data_15[perm_data_15["EMPLOYER_NAME"] == 'MAJESCOMASTEK']
#d4[d4.duplicated(subset=['DECISION_DATE', 'CASE_RECEIVED_DATE', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'JOB_INFO_WORK_CITY', 'COUNTRY_OF_CITIZENSHIP', 'EMPLOYER_NAME' , 'PW_JOB_TITLE_9089', 'PW_LEVEL_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_UNIT_OF_PAY_9089'], keep=False)]

CASE_NUMBER DECISION_DATE        CASE_STATUS CASE_RECEIVED_DATE  \
16 39234  A-15181-93531    2015-12-10  Certified-Expired         2015-06-30   
   39243  A-15181-93540    2015-12-09  Certified-Expired         2015-06-30   
   39270  A-15181-93589    2015-12-10  Certified-Expired         2015-06-30   
   39273  A-15181-93592    2015-12-10  Certified-Expired         2015-06-30   
   39275  A-15181-93594    2015-12-10  Certified-Expired         2015-06-30   

                           EMPLOYER_NAME          PW_JOB_TITLE_9089  \
16 39234     AMERIFOODS ENTERPRISE, INC.    Food Preparation Worker   
   39243  ULTIMO SOFTWARE SOLUTIONS INC.  Senior Programmer Analyst   
   39270    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   
   39273    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   
   39275    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   

         PW_LEVEL_9089 WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089  \
16 39234       Level I                16890                        Year   
   39243      Level II               109762                        Year   
   39270       Level I                17077                        Year   
   39273       Level I                17077                        Year   
   39275       Level I                17077                        Year   

         JOB_INFO_WORK_CITY JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP  \
16 39234       Fayetteville      NORTH CAROLINA            SOUTH KOREA   
   39243           San Jose          CALIFORNIA                  INDIA   
   39270          Rose Hill      NORTH CAROLINA            SOUTH KOREA   
   39273          Rose Hill      NORTH CAROLINA            SOUTH KOREA   
   39275          Rose Hill      NORTH CAROLINA            SOUTH KOREA   

         CLASS_OF_ADMISSION  TIME_TAKEN  
16 39234         Not in USA         163  
   39243               H-1B         162  
   39270         Not in USA         163  
   39273         Not in USA         163  
   39275         Not in USA         163

** Data Exploration: What countries are people applying from?**

The first set of question I was really interested in was:

1. What are the nationalities of the people who are applying for jobs, and how are the applications distributed? 
2. Where are the most number of applications coming form?
3. Does it take longer to process applications from countries that have more number of people applying?

I grouped the applications by country of citizenship, and calculated the Mean, Median and Standard Deviation of the time taken to process applications from each country. If there was only 1 application from a country, I set those standard deviations to be zero.

In [8]:
gbcountries = perm_data_15.groupby(['COUNTRY_OF_CITIZENSHIP']).size().reset_index(name='count')
pd.set_option('display.max_rows', len(gbcountries)) # display more number of rows than default
gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'] = ""

gbcountries["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN']
gbcountries["MEAN_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN']
gbcountries["STD_TIME_TAKEN"] = perm_data_15.groupby('COUNTRY_OF_CITIZENSHIP')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbcountries['STD_TIME_TAKEN'] = gbcountries['STD_TIME_TAKEN'].fillna(0)

**More Data Cleaning: Countries**
	
I wanted to visualize the countries people were applying from on a world map. To display the number of applications by country on a world map – I needed the 3 digit country codes.  I used the ISO 3166 library of country codes. 

Some of the countries that applicants said they were from did not exist in the ISO 3166 library (e.g. Yugoslavia, which split after 1991). Moreover, there were some differences in spelling (e.g. Macau / Macao) which had to be corrected. I did this by correcting each country that threw up an error, but there might be more efficient ways to do this using some form of NLP.

In [9]:
for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):
    if c == 'IVORY COAST':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('IVORY') == False]
    elif c == 'KOSOVO':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('KOSOVO') == False]
    elif c == 'PALESTINIAN TERRITORIES':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('PALESTINIAN') == False]
    elif c == 'DEMOCRATIC REPUBLIC OF CONGO':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('DEMOCRATIC REPUBLIC OF CONGO') == False]
    elif c == 'YUGOSLAVIA':
        gbcountries = gbcountries[gbcountries['COUNTRY_OF_CITIZENSHIP'].str.contains('YUGOSLAVIA') == False]

gbcountries.index = range(len(gbcountries))
    
for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):    
    if c == "BOLIVIA":
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "BOL")
    elif c == 'BRITISH VIRGIN ISLANDS':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "VG")
    elif c == 'BURMA (MYANMAR)':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "MYANMAR")
    elif c == "COTE d'IVOIRE":
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "CI")
        gbcountries.set_value(i, "count", 7)
    elif c == 'REPUBLIC OF CONGO':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "CONGO")
        gbcountries.set_value(i, "count", 5)
    elif c == 'IRAN':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "ir")
    elif c == 'MACAU':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", "MACAO")
    elif c == 'MACEDONIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'MK')
    elif c == 'MOLDOVA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'MDA')
    elif c == 'PALESTINE':
        gbcountries.set_value(i, "count", 23)
    elif c == 'RUSSIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'RUS')
    elif c == 'SERBIA AND MONTENEGRO':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'SERBIA')
    elif c == 'SOUTH KOREA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'KOR')
    elif c == 'ST KITTS AND NEVIS':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'KN')
    elif c == 'ST LUCIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'LC')
    elif c == 'SYRIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'SY')
    elif c == 'TANZANIA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'TZ')
    elif c == 'UNITED STATES OF AMERICA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'UNITED STATES')
    elif c == 'VENEZUELA':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'VEN')
    elif c == 'VIETNAM':
        gbcountries.set_value(i, "COUNTRY_OF_CITIZENSHIP", 'VIET NAM')
        
for i, c in enumerate(gbcountries['COUNTRY_OF_CITIZENSHIP']):
    gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'][i] = countries.get(c).alpha3

C:\Users\Sneha\Desktop\Anaconda2\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



**Data Visualization: Countries**

I plotted a world map of the number of applications by country. I use plotly for data visualization. I first set the purple color scale using the seaborn color palette and convert it into a format that works with plotly.


In [10]:
scl = map(str,(np.array(sns.cubehelix_palette(15, start=0, rot=0, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 15).tolist()
clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] 
data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'], z = gbcountries['count'],text = gbcountries['text'], marker = dict(line = dict (color = 'rgb(220,220,220)',width = 1)),  colorbar = dict( title = "Number of applications"))]
layout = dict(title = 'Number of Applications by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')),  margin = dict(l=0,r=0,b=0,t=100, pad=4))
fig1 = dict(data=data, layout=layout)
iplot(fig1)
#py.plot(fig1, filename='privacy-public', sharing='public')


This world map shows the number of applications by country. [(Click here for plot)](https://plot.ly/~sneharavi15/204/number-of-applications-by-country-hover-for-breakdown/)

Most of the applications come from citizens of India, followed by citizens of China. It also looks like there is either no data, or there are no applications from Central Africa. You can hover over the map to get the name of the country and the number of applications.

Next, I compute the percentage of applications from each country.


In [11]:
countperc = (gbcountries['count']/gbcountries['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 0.9])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(18)
for c in gbcountries.loc[b]['COUNTRY_OF_CITIZENSHIP']:
    if c == "KOR":
        topcount.append("SOUTH KOREA")
    else:
        topcount.append(c)
topcount.append('OTHERS')

fig2 = {'data': [{'labels': topcount,'values': topperc,'type': 'pie','hoverinfo':'none', 'textinfo':'label+percent'}],'layout': {'title': 'Percentage of Applications from Top 8 Countries'}}
iplot(fig2)
#py.plot(fig2, filename='pie-chart-countries', sharing='public')


I made a pie chart of the top 8 countries applicants come from. [(Click here for plot)](https://plot.ly/~sneharavi15/216/percentage-of-applications-from-top-8-countries/)

India takes up a large percentage, followed by China and South Korea, then Canada, Mexico, Philippines, UK and Taiwan.

Since pie charts may not be the best way to display lots of data, I will also visualize the data using a bar plot.

In [12]:
countperc = (gbcountries['count']/gbcountries['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 0.8])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(18)
for c in gbcountries.loc[b]['COUNTRY_OF_CITIZENSHIP']:
    if c == "KOR":
        topcount.append("SOUTH KOREA")
    else:
        topcount.append(c)
topcount.append('OTHERS')

trace0 = Bar(x=topcount,y=topperc,marker=dict(color='rgb(26, 118, 255)',line=dict(color='rgb(8,48,107)',width=1,)),opacity=0.7)

data = [trace0]
layout = Layout(xaxis=dict(tickangle=-45, tickfont=dict(size=12,color='rgb(107, 107, 107)')),
                 yaxis=dict(title='Percentage of Applications',titlefont=dict(size=16,color='rgb(107, 107, 107)'), tickfont=dict(size=14,color='rgb(107, 107, 107)')),
                title='Percentage of Applicaions from Top 12 Countries',
                barmode='group',bargap=0.4,bargroupgap=0.1)
fig2a = Figure(data=data, layout=layout)
iplot(fig2a, filename='applications-countries-bar')
#py.plot(fig2a, filename='bar-chart-countries', sharing='public')


This is a bar plot visualizing the top 12 countries with the highest number of applications. [(Click here for plot)](https://plot.ly/~sneharavi15/218/percentage-of-applicaions-from-top-12-countries/)

I wondered why there was such a large number of applications from India compared to other countries. I had already removed all the duplicates, but was there something I was missing? While looking through the duplicates data, I had found some applications with different case numbers, but they had exactly the same details in all the other columns. I was worried that applicants might be filing multiple applications just to increase their chances of getting a visa.

I decided to investigate this further, and accessed additional details about these application from this website: http://dolstats.com/. I found that there were differences in these applications: for example, they had differing levels of education, and where they were educated were different, so I concluded that they were different persons. 
Moreover, the DOL website stated that it "will no longer allow for such multiple filings under the PERM Program" in order to streamline the process and reduce the processing time.

Maybe the companies got a lot of applications around that time, or filed multiple applications together around the same time.


In [13]:
d3 = perm_data_15[perm_data_15.duplicated(subset=['DECISION_DATE', 'CASE_RECEIVED_DATE', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'JOB_INFO_WORK_CITY', 'COUNTRY_OF_CITIZENSHIP', 'EMPLOYER_NAME' , 'PW_JOB_TITLE_9089', 'PW_LEVEL_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_UNIT_OF_PAY_9089'], keep=False)]
d3.tail()
#d4 = perm_data_15[perm_data_15["EMPLOYER_NAME"] == 'MAJESCOMASTEK']
#d4[d4.duplicated(subset=['DECISION_DATE', 'CASE_RECEIVED_DATE', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'JOB_INFO_WORK_CITY', 'COUNTRY_OF_CITIZENSHIP', 'EMPLOYER_NAME' , 'PW_JOB_TITLE_9089', 'PW_LEVEL_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_UNIT_OF_PAY_9089'], keep=False)]

CASE_NUMBER DECISION_DATE        CASE_STATUS CASE_RECEIVED_DATE  \
16 39234  A-15181-93531    2015-12-10  Certified-Expired         2015-06-30   
   39243  A-15181-93540    2015-12-09  Certified-Expired         2015-06-30   
   39270  A-15181-93589    2015-12-10  Certified-Expired         2015-06-30   
   39273  A-15181-93592    2015-12-10  Certified-Expired         2015-06-30   
   39275  A-15181-93594    2015-12-10  Certified-Expired         2015-06-30   

                           EMPLOYER_NAME          PW_JOB_TITLE_9089  \
16 39234     AMERIFOODS ENTERPRISE, INC.    Food Preparation Worker   
   39243  ULTIMO SOFTWARE SOLUTIONS INC.  Senior Programmer Analyst   
   39270    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   
   39273    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   
   39275    HOUSE OF RAEFORD FARMS, INC.            Poultry Trimmer   

         PW_LEVEL_9089 WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089  \
16 39234       Level I                16890                        Year   
   39243      Level II               109762                        Year   
   39270       Level I                17077                        Year   
   39273       Level I                17077                        Year   
   39275       Level I                17077                        Year   

         JOB_INFO_WORK_CITY JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP  \
16 39234       Fayetteville      NORTH CAROLINA            SOUTH KOREA   
   39243           San Jose          CALIFORNIA                  INDIA   
   39270          Rose Hill      NORTH CAROLINA            SOUTH KOREA   
   39273          Rose Hill      NORTH CAROLINA            SOUTH KOREA   
   39275          Rose Hill      NORTH CAROLINA            SOUTH KOREA   

         CLASS_OF_ADMISSION  TIME_TAKEN  
16 39234         Not in USA         163  
   39243               H-1B         162  
   39270         Not in USA         163  
   39273         Not in USA         163  
   39275         Not in USA         163

I tried to double check this by plotting the number of applications from India and China over the years, and checking whether applications from India consistently made up ~50% of all applications over all the years/

I loaded the applications from 2008 - 2016, dropped null values, dropped the "Withdrawn" cases and the case number duplicates

In [14]:
# All Applications

perm_data = pd.read_pickle('perm')

print(str(len(perm_data)) + ' Total Applications')
print("We don't want the cases where they withdrew the application because they were not processed and it doesn't make sense to take them into account.")
perm_data = perm_data[perm_data['CASE_STATUS'].str.startswith('W') == False]
print(str(len(perm_data)) + ' Total Applications Processesd')
perm_data = perm_data.dropna(subset=['CASE_NUMBER', 'DECISION_DATE', 'CASE_STATUS', 'EMPLOYER_NAME', 'PW_JOB_TITLE_9089', 'PW_LEVEL_9089', 'WAGE_OFFER_FROM_9089', 'WAGE_OFFER_UNIT_OF_PAY_9089', 'JOB_INFO_WORK_CITY','JOB_INFO_WORK_STATE', 'COUNTRY_OF_CITIZENSHIP', 'CLASS_OF_ADMISSION'], how='any')
print(str(len(perm_data)) + ' applications after nulls dropped')
perm_data = perm_data.drop_duplicates(subset=['CASE_NUMBER'], keep = 'last')
print(str(len(perm_data)) + ' applications after duplicates dropped')
print(str(len(perm_data[perm_data['CASE_STATUS'].str.startswith("C")])) + " applications were accepted and " + str(len(perm_data[perm_data['CASE_STATUS'].str.startswith("D")]))+ " applications were denied.")
#print(perm_data.isnull().sum())

622637 Total Applications
We don't want the cases where they withdrew the application because they were not processed and it doesn't make sense to take them into account.
593055 Total Applications Processesd
513316 applications after nulls dropped
510750 applications after duplicates dropped
469496 applications were accepted and 41254 applications were denied.


I then plotted the total number of applications processed each year, total no of applications processed from India each year, and from China each year

In [15]:
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

appsperyear = perm_data.groupby([pd.DatetimeIndex(perm_data['DECISION_DATE']).year])
appsperyear.count()['CASE_NUMBER'][1:]

appsperyearindia = perm_data['CASE_NUMBER'].groupby([perm_data['COUNTRY_OF_CITIZENSHIP']=="INDIA", perm_data['DECISION_DATE'].dt.year])
#appsperyearindia.size()
#print(A[A.index.levels[1]==2007])
appsperyearchina = perm_data['CASE_NUMBER'].groupby([perm_data['COUNTRY_OF_CITIZENSHIP']=="CHINA", perm_data['DECISION_DATE'].dt.year])
                      
years = np.linspace(2008, 2016, 9).tolist()

trace1 = Scatter(x = years,y = appsperyear.count()['CASE_NUMBER'][1:],mode = 'lines+markers', name = 'total number of apps')
trace2 = Scatter(x = years,y = appsperyearindia.size().loc[True,][1:],mode = 'lines+markers', name = 'number of apps from India')
trace3 = Scatter(x = years,y = appsperyearchina.size().loc[True,][1:],mode = 'lines+markers', name = 'number of apps from China')

layout= Layout(title= 'Number of applications over the years (2008-2016)', xaxis= dict(title= 'Years',), yaxis=dict( title= 'Number of Applications',))
fig0= Figure(data=[trace1, trace2, trace3], layout=layout)
iplot(fig0, filename='trend in apps over years-2')
#py.plot(fig0, filename='trend in apps over years-2', sharing='public')

The number of applications from India does seem to make up ~50% of the total number ofapplications every year. The number of applications from China are much less, so it doesn't seem like there was anomalous data in 2015.  [(Click here for plot)](https://plot.ly/~sneharavi15/234/number-of-applications-over-the-years-2008-2016/)

I also wondered why citizens of the US were applying for permanent work visas. I investigated this a little bit and found out that these citizens were born in the US and had citizenships, but it seemed like they had not lived in the US, so they had to apply for a permanent work visa.


Next, I look at the mean time it takes to process applications from each country, and whether it is related to the number of applications from each country. Again, I set the green color scale using the seaborn color palette and convert it into a format that works with plotly.

In [16]:
scl = map(str,(np.array(sns.cubehelix_palette(6, start=0, rot=1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 6).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] + '<br>' +\
    'Mean time '+ gbcountries['MEAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbcountries['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbcountries['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'],z = gbcountries['MEAN_TIME_TAKEN'],text = gbcountries['text'],marker = dict(line = dict (color = 'rgb(255,255,255)', width = 2)),colorbar = dict( title = "Mean time taken" )) ]
layout = dict(title = 'Mean time taken by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')),  margin = dict(l=0,r=0,b=0,t=100, pad=4))

fig3 = dict(data=data, layout=layout)
iplot(fig3)
#py.plot(fig3, filename='mean-time-countries', sharing='public')

This map shows the mean time taken to process applications from each country (Hover for standard deviations). [(Click here for plot)](https://plot.ly/~sneharavi15/220/mean-time-taken-by-country-hover-for-breakdown/)

It does not look like the number of days needed to process an application (green world map) depends on the number of applications from that country (purple world map). 

However, some countries have very large standard deviations, implying that there might be outliers, and that the median might be a better estimate. Let’s see if the median processing time depends on the number of applications. 

In [17]:
scl = map(str,(np.array(sns.cubehelix_palette(6, start=0, rot=1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 6).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbcountries['text'] = gbcountries['COUNTRY_OF_CITIZENSHIP'] + '<br>' +\
    'Median time '+ gbcountries['MEDIAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
          'No apps ' + gbcountries['count'].astype(str)


data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbcountries['COUNTRY_OF_CITIZENSHIP_ABBR'],z = gbcountries['MEDIAN_TIME_TAKEN'],text = gbcountries['text'], marker = dict(line = dict (color = 'rgb(255,255,255)',width = 2 )),colorbar = dict(title = "Median time taken")) ]

layout = dict(title = 'Median time taken by Country<br>(Hover for breakdown)',geo = dict(showframe = False,showcoastlines = False,projection = dict(type = 'Mercator')), margin = dict(l=0,r=0,b=0,t=100, pad=4))

fig4 = dict(data=data, layout=layout)
iplot(fig4, filename='median-time-world-map')
#py.plot(fig4, filename='median-time-countries', sharing='public')

The median times are definitely more consistent than the mean times, however, it still does not look like the processing time depends on the number of applications. [(Click here for plot)](https://plot.ly/~sneharavi15/222/median-time-taken-by-country-hover-for-breakdown/)

**Statistical Analysis: Countries**

To confirm this, I calculate the correlation co-efficient. The correlation coefficient is 0.0185 which shows that there is no correlation between time taken and number of applications. 


In [18]:
round(np.corrcoef(gbcountries['count'],  gbcountries['MEAN_TIME_TAKEN'])[0, 1].astype(float),4)

0.0187

You can see this better when looking at the data on a scatter plot. I split the data up into two scatter plots so it is easier to see that there is no trend. The first plot has all countries with < 1000 applications, and the second has all countries with > 1000 applications. 

In [19]:
err = ((gbcountries['STD_TIME_TAKEN'][gbcountries['count'] < 1000])/(gbcountries['count'][gbcountries['count'] < 1000]**(1/2.0)))*1.96
trace = Scatter(x = gbcountries['count'][gbcountries['count'] < 1000],y = gbcountries['MEAN_TIME_TAKEN'][gbcountries['count'] < 1000].astype(int),mode = 'markers',error_y=dict(type='data',array=err.astype(int),color='#85144B',thickness=0.5,width=2, opacity=0.4))
layout= Layout(title= 'No of days taken to process <1000 applications',xaxis= dict(title= 'Number of applications per country',zeroline = True,),yaxis=dict(title= 'Number of days',zeroline = True,),showlegend= False)
fig5= Figure(data=[trace], layout=layout)
iplot(fig5, filename='scatter')
#py.plot(fig5, filename='scatter-processing-time-<1000', sharing='public')

I plot the mean time taken for each country (blue) and the standard error (pink). It does not seem like the processing time increases with the number of applications. [(Click here for plot)](https://plot.ly/~sneharavi15/232/no-of-days-taken-to-process-1000-applications/)

In [20]:
err = ((gbcountries['STD_TIME_TAKEN'][gbcountries['count'] >= 1000])/(gbcountries['count'][gbcountries['count'] >= 1000]**(1/2.0)))*1.96
trace = Scatter( x = gbcountries['count'][gbcountries['count'] >= 1000], y = gbcountries['MEAN_TIME_TAKEN'][gbcountries['count'] >= 1000].astype(int), mode = 'markers',error_y=dict(type='data',array=err.astype(int),color='#85144B', thickness=0.5, width=2, opacity=0.4))
layout= Layout(title= 'Mean and Standard error of number of days taken to process >1000 applications',xaxis= dict(title= 'Number of applications',),yaxis=dict(title= 'Number of days',),showlegend= False)
fig6= Figure(data=[trace], layout=layout)
iplot(fig6, filename='scatter')
#py.plot(fig6, filename='mean-time-scatter->1000', sharing='public')

The same is true for the processing times for countries with > 1000 applications. [(Click here for plot)](https://plot.ly/~sneharavi15/224/mean-and-standard-error-of-number-of-days-taken-to-process-1000-applications/)

**Data Exploration: States**

Since the permanent work visa applications are filed only after applicants receive job offers from employers in the US, the second set of questions I was really interested in was:

1. How are the job offers distributed across the 50 US states?
2. Which states are most of the job offers coming from?
3. Does it take longer to process applications in states where more number of people are receiving offers?

I grouped the job offers by US states, and calculated the Mean, Median and Standard Deviation of the time taken to process applications from each state. If there was only 1 application from a state, I set those standard deviations to be zero.

In [21]:
gbstates = perm_data_15.groupby(['JOB_INFO_WORK_STATE']).size().reset_index(name='count')

gbstates['JOB_INFO_WORK_STATE_ABBR'] = ""

gbstates["MEAN_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].mean().reset_index()['TIME_TAKEN']
gbstates["MEDIAN_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].median().reset_index()['TIME_TAKEN']
gbstates["STD_TIME_TAKEN"] = perm_data_15.groupby('JOB_INFO_WORK_STATE')['TIME_TAKEN'].std().reset_index()['TIME_TAKEN']
gbstates['STD_TIME_TAKEN'] = gbstates['STD_TIME_TAKEN'].fillna(0)

**More Data Cleaning: States**

I corrected any spelling mismatches in the states, and dropped all territories to make plotting the maps easier. I also converted all state names to their 2 letter abbreviations to display them on a US map.

In [22]:
# only keep 50 states to make plotting maps easier
todrop = [ 'FEDERATED STATES OF MICRONESIA', 'GUAM', 'MARSHALL ISLANDS','NORTHERN MARIANA ISLANDS', 'PUERTO RICO', 'VIRGIN ISLANDS']
for d in todrop:
    gbstates = gbstates[gbstates['JOB_INFO_WORK_STATE'] != d]
gbstates.index = range(len(gbstates))

# Create state abbreviations
for i, p in enumerate(gbstates['JOB_INFO_WORK_STATE']):
    if p == "UTAH":
        gbstates.set_value(i, "JOB_INFO_WORK_STATE", "UT")
    gbstates['JOB_INFO_WORK_STATE_ABBR'][i] = str(us.states.lookup(unicode(gbstates['JOB_INFO_WORK_STATE'][i])).abbr) 

C:\Users\Sneha\Desktop\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



**Data Visualization: States**

The US map below shows the number of job offers by state.


In [23]:
# set the colormap
scl = map(str,(np.array(sns.cubehelix_palette(15, start=0, rot=0, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 15).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' 

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'],z = gbstates['count'],locationmode = 'USA-states',text = gbstates['text'],marker = dict(line = dict (color = 'rgb(255,255,255)',width = 2)),colorbar = dict(title = "Number of applications" )) ]

layout = dict(title = 'Number of Jobs / Applications by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'),  margin = dict(l=0,r=0,b=0,t=100, pad=4))
fig7 = dict(data=data, layout=layout)
iplot(fig7, filename='apps-by-state-map')
#py.plot(fig7, filename='state-apps', sharing='public')

Most of the job offers are from California, followed by Texas. You can hover over the map to get the name of the state and the number of applications for jobs. [(Click here for plot)](https://plot.ly/~sneharavi15/210/number-of-jobs-applications-by-state-hover-for-breakdown/)

Next, I compute the percentage of offers from each state.

In [24]:
countperc = (gbstates['count']/gbstates['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 2.9])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(int(100-sum(a[b])))
for c in gbstates.loc[b]['JOB_INFO_WORK_STATE_ABBR']:
        topcount.append(c)
topcount.append('OTHER STATES')

fig8 = {'data': [{'labels': topcount, 'values': topperc, 'type': 'pie','hoverinfo':'none','textinfo':'label+percent'}],'layout': {'title': 'Percentage of Applications from top 9 States'} }

iplot(fig8)
#py.plot(fig8, filename='state-pie-chart', sharing='public')

I made a pie chart of the top 9 states with the most job offers. [(Click here for plot)](https://plot.ly/~sneharavi15/212/percentage-of-applications-from-top-9-states/)

California and Texas take up large percentages, followed by New Jersey, New York, Illinois, Massachusetts, Washington, Virginia and Michigan.

I also make a bar chart to visualize the data better.

In [25]:
countperc = (gbstates['count']/gbstates['count'].sum())*100
a =  countperc.sort_values(ascending = False)
b = np.array(countperc).argsort()[::-1][:len(a[a > 2.3])]

topcount = []
topperc = []
for val in a[b]:
    topperc.append(val)
topperc.append(int(100-sum(a[b])))
for c in gbstates.loc[b]['JOB_INFO_WORK_STATE_ABBR']:
        topcount.append(c)
topcount.append('OTHER STATES')

trace0 = Bar(x=topcount,y=topperc,marker=dict(color='rgb(26, 118, 255)',line=dict(color='rgb(8,48,107)',width=1,)),opacity=0.7)

data = [trace0]
layout = Layout(xaxis=dict(tickangle=-45, tickfont=dict(size=12,color='rgb(107, 107, 107)')),
                 yaxis=dict(title='Percentage of Applications',titlefont=dict(size=16,color='rgb(107, 107, 107)'), tickfont=dict(size=14,color='rgb(107, 107, 107)')),
                title='Percentage of Applicaions from Top 12 States',
                barmode='group',bargap=0.4,bargroupgap=0.1)
fig8a = Figure(data=data, layout=layout)
iplot(fig8a, filename='applications-states-bar')
#py.plot(fig8a, filename='state-bar-chart', sharing='public')

This is a bar chart showing the top 12 states applications are sent to. [(Click here for plot)](https://plot.ly/~sneharavi15/214/percentage-of-applicaions-from-top-12-states/)

The applications seem to be concentrated around states in the East and West Coast.

Next, I look at the mean time taken to process applications from each state. Do California and Texas have the longest processing times?

In [26]:
scl = map(str,(np.array(sns.cubehelix_palette(20, start=0, rot=-1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 20).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' +\
    'Mean time '+ gbstates['MEAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbstates['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbstates['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'], z = gbstates['MEAN_TIME_TAKEN'],text = gbstates['text'], locationmode = 'USA-states', marker = dict(line = dict ( color = 'rgb(255,255,255)', width = 2 )), colorbar = dict(  title = "Mean time taken")) ]
layout = dict(title = 'Mean time taken by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'), margin = dict(l=0,r=0,b=0,t=100, pad=4))

fig9 = dict(data=data, layout=layout)
iplot(fig9, filename='state-map-time')
#py.plot(fig9, filename='mean-time-state-map', sharing='public')

This map plots the mean time taken to process applications for each state (Hover for standard deviations). [(Click here for plot)](https://plot.ly/~sneharavi15/226/mean-time-taken-by-state-hover-for-breakdown/)

It looks like the time taken to process applications for certain states might be higher because there are more job offers? For example, the number of job offers in Texas is high, and the processing time for those offers is also high. The opposite is true for Wyoming – where there were only 20 job offers, and the mean processing time is low. 

Again, to ensure that outliers are not affecting the data too much, I also plot the median time for each state. 

In [27]:
scl = map(str,(np.array(sns.cubehelix_palette(20, start=0, rot=-1, dark=0.2, light=0.95))*255).astype(int))
scl = [w.replace('[ ','rgb(') for w in scl]
scl = [w.replace('[','rgb(') for w in scl]
scl= [w.replace(']',')') for w in scl]
scl= [w.replace('  ',' ') for w in scl]
scl= [w.replace(' ',',') for w in scl]
fr = np.linspace(0.0, 1.0, 20).tolist()

clrscl = [list(a) for a in zip(fr, scl)]

gbstates['text'] = gbstates['JOB_INFO_WORK_STATE_ABBR'] + '<br>' +\
    'Median time '+ gbstates['MEDIAN_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
     'Std '+ gbstates['STD_TIME_TAKEN'].astype(int).astype(str) + '<br>' +\
      'No apps ' + gbstates['count'].astype(str)

data = [ dict(type='choropleth',colorscale = clrscl,autocolorscale = False,locations = gbstates['JOB_INFO_WORK_STATE_ABBR'],z = gbstates['MEDIAN_TIME_TAKEN'],text = gbstates['text'],locationmode = 'USA-states', marker = dict(line = dict (color = 'rgb(255,255,255)',width = 2 ) ), colorbar = dict( title = "Median time taken" )) ]

layout = dict(title = 'Median time taken by State<br>(Hover for breakdown)',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,lakecolor = 'rgb(255, 255, 255)'),  margin = dict(l=0,r=0,b=0,t=100, pad=4))

fig10 = dict(data=data, layout=layout)
iplot(fig10, filename='state-map-median-time')
#py.plot(fig10, filename='median-time-state-map', sharing='public')

The median times look pretty similar to the means, so we will focus on the mean times for each state. [(Click here for plot)](https://plot.ly/~sneharavi15/228/median-time-taken-by-state-hover-for-breakdown/)

You might be able to see if there is a trend more clearly by looking at the data on a scatter plot – the means for each state are in blue and the standard errors are in pink. 

In [28]:
err = ((gbstates['STD_TIME_TAKEN'])/(gbstates['count']**(1/2.0)))*1.96

trace = Scatter( x = gbstates['count'], y = gbstates['MEAN_TIME_TAKEN'].astype(int), mode = 'markers',error_y=dict(type='data',array=err.astype(int),color='#85144B', thickness=0.5,width=2,opacity=0.4 ))
layout= Layout(title= 'No of days taken to process applications by state',xaxis= dict(title= 'Number of applications',zeroline = True, ), yaxis=dict(title= 'Number of days', zeroline = True,), showlegend= False)

fig11= Figure(data=[trace], layout=layout)
iplot(fig11, filename='scatter state')
#py.plot(fig11, filename='scatter-states', sharing='public')

**Statistical Analysis: Countries**

It looks like there might be a slightly positive trend. [(Click here for plot)](https://plot.ly/~sneharavi15/230/no-of-days-taken-to-process-applications-by-state/)

To quantify the trend, I calculate the correlation co-efficient. The correlation coefficient is 0.1913, which shows that there is a small correlation between the time taken and number of jobs in each state. 

In [29]:
round(np.corrcoef(gbstates['count'],  gbstates['MEAN_TIME_TAKEN'])[0, 1].astype(float),4)

0.1885

**Data Exploration: Accepted vs Denied Applications**

From the previous set of analyses, we know that where a person is from, or where he has a job offer, do not have a significant impact on processing time. 

I wondered if other factors might affect the processing time. For example, I would imagine that applications would be accepted quicker than they would be denied, especially if there are certain preset criteria to accept an application.

And so, the question I try to answer here is: Is there a difference in the time taken to deny vs accept applications? Do applications that get denied have longer processing times than those that are accepted?

To answer this question, I split the data into applications that were accepted vs those that were denied.  

I visualized the time taken to process these 2 categories of applications using a box plot, and then performed a 2-sample t-test with unequal variance.


In [10]:
timeaccep = perm_data_15['TIME_TAKEN'][perm_data_15['CASE_STATUS'].str.startswith("C")]
timeden = perm_data_15['TIME_TAKEN'][perm_data_15['CASE_STATUS'].str.startswith("D")]

trace0 = Box(y=timeaccep,name = 'Accepted Applications', boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
trace1 = Box(y=timeden, name = 'Denied Applications',boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
layout = Layout(title='Distribution of time taken for Accepted vs Denied Applications',yaxis=dict(title= 'Number of Days'))
fig13 =Figure(data=[trace0, trace1], layout=layout)
iplot(fig13)
#py.plot(fig13, filename='box-plot-accept-deny', sharing='public')

The box plot on the left (blue) shows the distribution of the processing time for Accepted Applications, while the box plot on the right (orange) shows the distribution of the processing times for Denied Applications.  [(Click here for plot)](https://plot.ly/~sneharavi15/236/distribution-of-time-taken-for-accepted-vs-denied-applications/)

The processing times look very different for the 2 groups - In fact, the interquartile range, median and mean are much smaller for accepted applications compared to those denied. The applications that were denied actually seem to take a longer time to get processed! Since the acceptance rate of getting these visas is actually very large (>90%), maybe the DOL checks the applications that are denied very thoroughly before issuing a denial.

Additionally, the points on the left show that the distribution of processing times are also very different. The processing times are distributed over a much wider range for applications that were denied. The processing times for accepted applications seem to fall within a smaller range, and it looks like there are two bands of processing times, one within 250 days and the other within 800 days (shown in histogram below). This is an interesting trend, and I do wonder how the applications differ in these 2 bands, I might look into this further in the future.


**Statistical Analysis: Accepted vs Denied Applications**

I then performed a statistical test to determine if this was a significant difference. I did a one-tailed t-test with unequal variance to determine quantitatively if denied applications were processed slower.

(This is the Welch's t-test: one-tailed independent 2-sample t-test with unequal sample size and variance)

Null Hypothesis: The processing times are identical for accepted vs denied applications 

Alternative Hypothesis: The processing times of denied applications are greater than those of accepted applications.

In [31]:
import scipy as sp

print("The p value is " + str(sp.stats.ttest_ind(timeden, timeaccep,axis=0, equal_var=False).pvalue/float(30)) + " and the t value is " + str(sp.stats.ttest_ind(timeden, timeaccep,axis=0, equal_var=False).statistic/float(2)))

def cohensd(val1,val2):
    return ( np.mean(val1) - np.mean(val2) ) / np.sqrt( (np.std(val1) ** 2 + np.std(val2) ** 2) / 2 )
print ("The effect size is " + str(cohensd(timeden, timeaccep)))
print("It takes " + str(int(np.mean(timeden))) + " days on average to deny an application")
print("It takes " + str(int(np.mean(timeaccep)))+ " days on average to accept an application")

The p value is 0.0 and the t value is 23.6546688556
The effect size is 0.971504357256
It takes 628 days on average to deny an application
It takes 264 days on average to accept an application


We reject the null hypothesis. The processing times of denied applications were greater than those for accepted applications, as seen by the very low p-value and high t value. The effect size was also large, and the means of the 2 groups differed by almost 1 standard deviation.

**Data Exploration: Level of Experience**

I wondered if other factors, like the level of experience the person has, might affect the processing time. The question I try to answer here is:

1) Does increased experience mean a faster processing time?

To answer this question, I grouped the data into low and high levels of experience, where low experience is defined as anyone who fell into the Level 1 and 2 categories (under PW_LEVEL_9089), and high is anyone who fell into Levels 3 and 4. 

I then visualized the data using a box plot.


In [32]:
gbexper = perm_data_15.groupby(['PW_LEVEL_9089'])#.size().reset_index(name='count')
low = gbexper.get_group('Level I')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist() + gbexper.get_group('Level II')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist()
high = gbexper.get_group('Level III')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist() + gbexper.get_group('Level IV')['TIME_TAKEN'].reset_index()['TIME_TAKEN'].tolist()


trace0 = Box(y=low,name = 'Low Experience', boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
trace1 = Box(y=high, name = 'High Experience',boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
layout = Layout(title='Distribution of time taken for Low vs High Experience',yaxis=dict(title= 'Number of Days'))
fig12 =Figure(data=[trace0, trace1], layout=layout)
iplot(fig12)

#py.plot(fig12, filename='box-plot', sharing='public')

The box plot above shows the distributions of processing times for low and high levels of experience. [(Click here for plot)](https://plot.ly/~sneharavi15/206/distribution-of-time-taken-for-low-vs-high-experience/)

The two distributions look somewhat similar, with a break in the distribution at around 300 days. However, the processing times for low levels of experience have a slightly longer tail. The mean processing time was also larger for lower levels of experience than higher levels of experience. Moreover, the interquartile range seemed a little bigger for low levels of experience. This shows that at least 75% of the data falls within a higher processing time range when the level of experience is lower.

**Statistical Analysis: Level of Experience**

I then performed a statistical test to determine quantitatively if a decreased level of experience meant that the application was processed slower. I did a one-tailed t-test with unequal variance (This is the Welch's t-test: one-tailed independent 2-sample t-test with unequal sample size and variance).

Null Hypothesis: There is no difference in processing time for applicants with a lower or higher level of experience. 

Alternative Hypothesis: There is a longer processing time for applicants with a lower level of experience, than those with a higher level of experience. 

In [33]:
import scipy as sp

print("The p value is " + str(sp.stats.ttest_ind(low, high,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(low, high,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(low, high)))
print("It takes " + str(int(np.mean(low))) + " days to process applications with low levels of experience")
print("It takes " + str(int(np.mean(high)))+ " days to process applications with high levels of experience")

The p value is 1.96389541794e-23 and the t value is 4.95446332581
The effect size is 0.069691972329
It takes 291 days to process applications with low levels of experience
It takes 276 days to process applications with high levels of experience


The p value of 2e-23 and t value > 0 suggests that we can reject the null hypothesis. The processing times of applications with low levels of experience were greater than those with higher levels of experience. 

However, the effect size was very small, which means that means of the 2 groups differed only by < 0.1 standard deviations. Hence the difference is trivial, even if it was statistically significant.

** Data Exploration: Processing times for 2016 versus 2015 applications**

Till now, we have been analyzing the data from 2015. The DOL office had stated that it had been working to reduce the processing time of visa applications. In line with this, I wondered if the processing times for 2016 had gotten quicker compared to 2015. I only have the data until the end of June 2016, so I decided to compare the processing times for Jan-June 2015 and Jan-June 2016.

Is there a difference in processing times in 2015 compared to 2016? Are applications in 2016 getting processed faster than those in 2015? I answer those questions below using a box plot, and doing a statistical test.

Another interesting question, also worth analyzing later : (Does increased experience mean higher wages? (using ANOVA, low value, paired t test, bonferroni correction))


In [34]:
timetaken = perm_data['DECISION_DATE'] - perm_data['CASE_RECEIVED_DATE']
timetaken = timetaken.fillna(value=0)
timetaken2 = (timetaken/np.timedelta64(1, 'D')).astype(int)
time2016 = timetaken2[pd.DatetimeIndex(perm_data['DECISION_DATE']).year == 2016]
time2015 = timetaken2[np.logical_and(pd.DatetimeIndex(perm_data['DECISION_DATE']).year == 2015, pd.DatetimeIndex(perm_data['DECISION_DATE']).month < 7)]

trace0 = Box(y=time2015, name = 'Applications: Jan- Jun 2015',boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
trace1 = Box(y=time2016,name = 'Applications: Jan-Jun 2016', boxmean=True, boxpoints='all', jitter = 0.3, pointpos = -2.5, marker=dict(size=2), line=dict(width=1))
layout = Layout(title='Distribution of time taken for 2015 vs 2016 Applications for same time range (Jan-Jun)',yaxis=dict(title= 'Number of Days'))
fig14 =Figure(data=[trace0, trace1], layout=layout)
iplot(fig14)
#py.plot(fig14, filename='box-plot-2015-2016', sharing='public')


The box plot on the left (blue) shows the distribution of the processing time for 2015 Applications, while the box plot on the right (orange) shows the distribution of the processing times for 2016 Applications.  [(Click here for plot)](https://plot.ly/~sneharavi15/238/distribution-of-time-taken-for-2015-vs-2016-applications-for-same-time-range-jan/)

The processing times look very different for the 2 groups - In fact, the interquartile range, median and mean are much smaller for applications processed in 2016 compared to those in 2015. The applications processed in 2016 do seem to take a shorter time to get processed! The DOL might be steadily clearing its backlogged applications every year which might improve overall processing times, or it might have implemented policies in place such as processing all applications only at the Atlanta Center instead of over many centers which might have shortened processing times. 

Additionally, the points on the left show that the distribution of processing times are also very different. The processing times are distributed over a much wider range for 2015 applications, and it looks like there are 2 or even 3 bands of processing times. The processing times for 2016 applications seem to fall within a smaller range, and it looks like there are also two bands of processing times, one within 500 days and the other within 800 days. Again, this is an interesting trend for both years, and I do wonder how the applications differ across these bands, I might look into this further in the future.


**Statistical Analysis: 2015 vs 2016 Applications**

I then performed a statistical test to determine if this was a significant difference. I did a one-tailed t-test with unequal variance to determine quantitatively if denied applications were processed slower.

(This is the Welch's t-test: one-tailed independent 2-sample t-test with unequal sample size and variance)

Null Hypothesis: There processing times are identical for 2015 vs 2016-processed applications 

Alternative Hypothesis: The processing times of applications from 2015 are greater than those of 2016.

In [35]:

print("The p value is " + str(sp.stats.ttest_ind(time2015, time2016,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(time2015, time2016,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(time2015, time2016)))
print("It takes " + str(int(np.mean(time2015))) + " days on average to process applications in 2015")
print("It takes " + str(int(np.mean(time2016)))+ " days on average to process applications in 2016")

The p value is 0.0 and the t value is 47.2975242189
The effect size is 0.675185193689
It takes 308 days on average to process applications in 2015
It takes 174 days on average to process applications in 2016


The low p value and t value > 0 suggests that we can reject the null hypothesis. The processing times of applications in 2015 were greater than those from 2016. The effect size was also relatively large, which means that the means of the 2 groups differed by > ~0.7 standard deviations. 

**Machine Learning: Predicting the time it takes to process the visa. **

Let's try to predict how long it will take for a person’s visa application to be processed, using ML algorithms. I will train the model on these features: 

1. applicant's nationality
2. the state their job is in
3. applicant's level of experience
4. visa class they are currently on.

Even though feature 1 did not have a significant impact on the processing time (as explored above), we only looked at the correlation coefficient. I am going to include this feature for now, in case there is something in the data I did not capture in the exploration phase. 

**Linear Regression**

First, I am going to try using linear regression to predict the exact number of days. There are a total of 50 states, 137 countries, 4 levels of experience and 48 visa classes. I use one hot encoding to pre-process my data since these are all categorical features.


In [36]:
from sklearn.preprocessing import OneHotEncoder

perm_data_trunc = perm_data_15

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_TAKEN'].reset_index()['TIME_TAKEN']

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
lm = LinearRegression()
x_train,x_test,y_train,y_test = train_test_split(xnew, y,test_size=0.2)

lm.fit(x_train, y_train)

print("Sum of squares error: %.2f"
      % np.mean((lm.predict(x_test) - y_test) ** 2))
print('R squared value: %.2f' % lm.score(x_test, y_test))
print("Median of test data: %.2f" % np.mean(y_test))
print("Median of prediction: %.2f" % np.median(lm.predict(x_test)))
print("Std dev of test data: %.2f" % np.std(y_test))
print("Std dev of prediction: %.2f" % np.std(lm.predict(x_test)))

Sum of squares error: 2199796056733076018325094400.00
R squared value: -52967389900065201979392.00
Median of test data: 284.34
Median of prediction: 282.83
Std dev of test data: 203.79
Std dev of prediction: 46900613142014.20


The SSE is very high and the R2 value is highly negative showing that the prediction by this model is very poor.

I also notice that the model is capturing the median of the data reasonably well, but not the standard deviation. 

I looked the difference between the predicted and actual values more closely. 

In [37]:
(lm.predict(x_test) - y_test)[0:10]

72421     51.156250
15247    152.203125
67783    129.640625
51050   -177.046875
34258   -245.656250
67075     74.546875
16493     96.328125
67818    100.203125
48639     41.828125
4602    -178.296875
Name: TIME_TAKEN, dtype: float64

I noticed that part of the reason the performance of linear regression was so poor was because there is a big difference between the prediction and actual value. This difference also varies widely for each value, meaning that there is a lot of variance in the data that the model is not accounting for. The number of days needed can vary by a few months, and so an exact prediction will not be very precise and the SSE will be very large. 

Thus, instead of trying to predict the exact number of days, I am going to try to predict whether the visas will be processed within 4 months, within 4-8 months ; 8 - 12 months; 12-16 months; 16-20 months or > 20 months using Logistic Regression. This will still be very useful to a potential applicant, when they check what time range their processing time could fall under, based on features in their application.

I am going to try Logistic Regression next, but here are some other ideas I also had for things to improve when using Linear Regression: define better features, regularization, try polynomial regression, add more data, try normalizing the y value (number of days) so it is between 0 and 1.

In [38]:
# Define categories:

perm_data_15.ix[perm_data_15["TIME_TAKEN"] >= 600, 'TIME_CATEGORIES'] = 6
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 600, 'TIME_CATEGORIES'] = 5
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 480, 'TIME_CATEGORIES'] = 4
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 360, 'TIME_CATEGORIES'] = 3
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 240, 'TIME_CATEGORIES'] = 2
perm_data_15.ix[perm_data_15["TIME_TAKEN"] < 120, 'TIME_CATEGORIES'] = 1

**Logistic Regression**

I defined the output categories above and implement logistic regression below:

I perform one hot encoding on the same set of features:

1. applicant's nationality (137 countries)
2. the state their job is in (50 states)
3. applicant's level of experience (4 levels)
4. visa class they are currently on. (47 types)


In [39]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

perm_data_trunc = perm_data_15#[perm_data_15["TIME_TAKEN"] < 800]

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))

from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
#print('F1 scores are' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))

Accuracy of model on test set is 70.6308874566 %
Weighted F1 score is 0.590497734938
Weighted Precision score is 0.574130980424
Weighted Recall score is 0.706308874566


C:\Users\Sneha\Desktop\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\Sneha\Desktop\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.



The Logistic Regression does a much better job than Linear Regression at predicting the processing times - the accuracy of the model is ~70%. Next, I am going to explore the data a little more to improve the performance of the model.

**Data Exploration & Improving Logistic Regression (1)**

First, I plot the distribution of the processing times to see if there are any outliers.

In [40]:
data = [Histogram(x=perm_data_15["TIME_TAKEN"])]
layout= Layout(title= 'Distribution of time taken to process visa',
xaxis= dict(title= 'Number of days',zeroline = True,),yaxis=dict(title= 'Frequency',zeroline = True,),showlegend= False)
fig =Figure(data=data, layout=layout)
iplot(fig, filename='days-histogram')
#py.plot(fig, filename='hist', sharing='public')

In [41]:
len(perm_data_15[perm_data_15["TIME_TAKEN"] < 600])/float(len(perm_data_15))

0.9555505974515345

The histogram reflects the fact that most of the data falls within 90 days. [(Click here for plot)](https://plot.ly/~sneharavi15/208/distribution-of-time-taken-to-process-visa/)

96%, to be precise. Let's try eliminating these outliers and see if we get a better prediction with Logistic Regression.

In [42]:
from sklearn.preprocessing import OneHotEncoder

perm_data_trunc = perm_data_15[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))



from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'


Accuracy of model on test set is 73.6736282267 %


The accuracy of the model improves by about 4%! Let's look at the confusion matrix to see how well the model does with true and false positives.

In [43]:
print metrics.confusion_matrix(y_test, y_pred)
print ('\n'+"Predicting everything as falling within 4-8 months will still give accuracy of "+ str((len([x for x in y_test if x == 2])/float(len(y_test)))*100))
#print('F1 scores are' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))

[[    0    76     0     2     2]
 [    1 11315     0    23     6]
 [    0   642     0     3     1]
 [    0  1583     0    19    12]
 [    0  1689     0    19    25]]

Predicting everything as falling within 4-8 months will still give accuracy of 73.5828252692
Weighted F1 score is 0.630359706284
Weighted Precision score is 0.635221993563
Weighted Recall score is 0.736736282267


I was worried that the model might predict all times as falling within the 4 – 8 month category. It doesn’t do that – it does predict processing times in other categories. The F1, Precision and Recall Scores also increase. However, there are still plenty of false positives in the 4-8 month category, and the model doesn't perform much better than predicting everything as falling within 4-8 months. This is probably because the dataset is quite skewed towards these values. I am going to explore if there are new features that might reduce the number of false positives.

**Data Exploration & Improving Logistic Regression (2)**

I looked at the data again, and it seemed like if the application was denied, it was usually processed in < 4 months. (Compare Orange (Denied) and Blue (Accepted) Histograms) Adding the case status as another feature could improve the performance of the model. However, this is not something potential applicants will know in advance when they key in details about their application.

In [44]:
perm_data_15[perm_data_15['TIME_CATEGORIES']==1].head()

CASE_NUMBER DECISION_DATE CASE_STATUS CASE_RECEIVED_DATE  \
15 79034  A-15134-76322    2015-07-07      Denied         2015-06-01   
   79044  A-15138-77412    2015-06-23      Denied         2015-05-18   
   79059  A-14358-37833    2015-02-18      Denied         2015-01-13   
   79119  A-14304-21633    2015-01-09      Denied         2014-11-01   
   79121  A-15180-92423    2015-07-29      Denied         2015-07-29   

                         EMPLOYER_NAME                  PW_JOB_TITLE_9089  \
15 79034  E-BUSINESS INTERNATIONAL INC   SOFTWARE DEVELOPER, APPLICATIONS   
   79044   HEXAWARE TECHNOLOGIES, INC.  Software Developers, Applications   
   79059                   INOVANT LLC  Software Developers, Applications   
   79119              GCB SERVICES LLC                 SENIOR RF ENGINEER   
   79121                FACEBOOK, INC.  Software Developers, Applications   

         PW_LEVEL_9089 WAGE_OFFER_FROM_9089 WAGE_OFFER_UNIT_OF_PAY_9089  \
15 79034      Level II            77,210.00                        Year   
   79044      Level IV           108,805.00                        Year   
   79059     Level III           110,000.00                        Year   
   79119      Level IV           123,000.00                        Year   
   79121      Level II           147,977.86                        Year   

         JOB_INFO_WORK_CITY JOB_INFO_WORK_STATE COUNTRY_OF_CITIZENSHIP  \
15 79034     EAST BRUNSWICK          NEW JERSEY                  INDIA   
   79044          Jamesburg          NEW JERSEY                  INDIA   
   79059             Austin               TEXAS                  INDIA   
   79119             MCLEAN            VIRGINIA                  INDIA   
   79121         Menlo Park          CALIFORNIA             LUXEMBOURG   

         CLASS_OF_ADMISSION  TIME_TAKEN  TIME_CATEGORIES  
15 79034               H-1B          36              1.0  
   79044               H-1B          36              1.0  
   79059               H-1B          36              1.0  
   79119              H-1B1          69              1.0  
   79121               H-1B           0              1.0

Implementing Logistic Regression again with additional set of features:

In [45]:
perm_data_15.ix[perm_data_15['CASE_STATUS'].str.startswith("C"), 'ACCEPTED_CLASS'] = '2'
perm_data_15.ix[perm_data_15['CASE_STATUS'].str.startswith("D"), 'ACCEPTED_CLASS'] = '1'

from sklearn.preprocessing import OneHotEncoder

perm_data_trunc = perm_data_15[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'ACCEPTED_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))
perm_dict['ACCEPTED_CLASS'] = dict(zip( np.unique(x['ACCEPTED_CLASS']) ,range(len(np.unique(x['ACCEPTED_CLASS'])))))



from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()

logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %' + '\n'

print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling within 4-8 months will still give accuracy of "+ str((len([x for x in y_test if x == 2])/float(len(y_test)))*100))

#print('F1 scores are' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))

Accuracy of model on test set is 73.7449734077 %

[[   12    10     1    20    25]
 [    2 11179     8    34    50]
 [    2   570     3    22    28]
 [    7  1440     6    88    82]
 [    6  1678    10    47    88]]
Predicting everything as falling within 4-8 months will still give accuracy of 73.1158386302
Weighted F1 score is 0.646631718029
Weighted Precision score is 0.637722992496
Weighted Recall score is 0.737449734077


The model does a little better – accuracy improves by ~0.5%.  There are more true positives in categories 1,3,4 and 5.  The F1, Precision and Recall Scores also increase. However, there are still plenty of false positives in the 4-8 month category, and so there is definitely room for improvement.

#### Implementing Random Forests

I wondered if Random Forests might perform better, because of how the model randomly subsamples the data. I am still working on tuning the parameters of this model.

In [46]:
from sklearn.ensemble import RandomForestClassifier

perm_data_trunc = perm_data_15[perm_data_15["TIME_TAKEN"] < 600]

feature_cols = ['COUNTRY_OF_CITIZENSHIP','PW_LEVEL_9089', 'CLASS_OF_ADMISSION', 'JOB_INFO_WORK_STATE', 'ACCEPTED_CLASS'] 
x = perm_data_trunc[feature_cols].reset_index()[feature_cols]
y = perm_data_trunc['TIME_CATEGORIES'].reset_index()['TIME_CATEGORIES'].tolist()

perm_dict = {}
perm_dict['COUNTRY_OF_CITIZENSHIP'] = dict(zip( np.unique(x['COUNTRY_OF_CITIZENSHIP']) ,range(len(np.unique(x['COUNTRY_OF_CITIZENSHIP'])))))
perm_dict['PW_LEVEL_9089'] = dict(zip( np.unique(x['PW_LEVEL_9089']) ,range(len(np.unique(x['PW_LEVEL_9089'])))))
perm_dict['CLASS_OF_ADMISSION'] = dict(zip( np.unique(x['CLASS_OF_ADMISSION']) ,range(len(np.unique(x['CLASS_OF_ADMISSION'])))))
perm_dict['JOB_INFO_WORK_STATE'] = dict(zip( np.unique(x['JOB_INFO_WORK_STATE']) ,range(len(np.unique(x['JOB_INFO_WORK_STATE'])))))
perm_dict['ACCEPTED_CLASS'] = dict(zip( np.unique(x['ACCEPTED_CLASS']) ,range(len(np.unique(x['ACCEPTED_CLASS'])))))

names = []
names = perm_dict['COUNTRY_OF_CITIZENSHIP'].keys()
names.extend(perm_dict['PW_LEVEL_9089'].keys())
names.extend(perm_dict['CLASS_OF_ADMISSION'].keys())
names.extend(perm_dict['JOB_INFO_WORK_STATE'].keys())
names.extend(perm_dict['ACCEPTED_CLASS'].keys())



from sklearn import linear_model, datasets, metrics
#from sklearn.cross_validation import train_test_split
x = x.replace(to_replace = perm_dict)
#print(x)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()


rf = RandomForestClassifier(n_estimators=10)
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %' + '\n'
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling within 4-8 months will still give accuracy of "+ str((len([x for x in y_test if x == 2])/float(len(y_test)))*100))


#print(precision_score(y_test, y_pred, average=None))
#print(recall_score(y_test, y_pred, average=None))  
#print('F1 scores are' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted'))) 

Accuracy of model on test set is 72.6877675444 %

[[   17    16     5    15    15]
 [   12 10838    96   188   132]
 [    6   510    87    23    19]
 [   10  1283    80   143    83]
 [   16  1513    91    98   122]]
Predicting everything as falling within 4-8 months will still give accuracy of 73.0704371514
Weighted F1 score is 0.658874600857
Weighted Precision score is 0.641646285372
Weighted Recall score is 0.726877675444


Looking at the confusion matrix, the accuracy using Random Forests is still around the same. However, the F1 and Precision Scores are better. 

I wanted to take a look at the Feature Importance. Which features were performing the best at predicting processing times? I computed how much each feature decreases the weighted impurity in a tree. There are 283 features, so I plotted the top 20 features in a bar plot.


In [53]:
fimp = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_),names),reverse=True)
#print(fimp[:10])
trace0 = Bar(x=[x[1] for x in fimp[:20]],y=[x[0] for x in fimp[:20]],marker=dict(color='rgb(26, 118, 255)',line=dict(color='rgb(8,48,107)',width=1,)),opacity=0.7)
data = [trace0]
layout = Layout(xaxis=dict(title = 'Features', tickangle=-45, tickfont=dict(size=12,color='rgb(107, 107, 107)')),
                 yaxis=dict(title='Mean Decrease Impurity Score',titlefont=dict(size=16,color='rgb(107, 107, 107)'), tickfont=dict(size=14,color='rgb(107, 107, 107)')),
                title='Feature Importance of Top 20 Features',
                barmode='group',bargap=0.4,bargroupgap=0.1)
fig16 = Figure(data=data, layout=layout)
iplot(fig16, filename='feature-importance-mean-impurity-bar')
py.plot(fig16, filename='bar-chart-feature-importance', sharing='public')

u'https://plot.ly/~sneharavi15/240'

Whether the application is accepted or denied is a good feature at predicting processing times, like expected from the statistical tests above. The level of experience also seems to be a good feature. Interestingly, the state of Pennsylvania as well as the O-2 visa class are also good feaures.

What are the 20 worst features, I wondered?

In [48]:
fimp = sorted(zip(map(lambda x: round(x, 7), rf.feature_importances_),names),reverse=True)
trace0 = Bar(x=[x[1] for x in fimp[-20:]],y=[x[0] for x in fimp[-20:]],marker=dict(color='rgb(26, 118, 255)',line=dict(color='rgb(8,48,107)',width=1,)),opacity=0.7)
data = [trace0]
layout = Layout(xaxis=dict(title = 'Features', tickangle=-45, tickfont=dict(size=12,color='rgb(107, 107, 107)')),
                 yaxis=dict(title='Mean Decrease Impurity Score',titlefont=dict(size=16,color='rgb(107, 107, 107)'), tickfont=dict(size=14,color='rgb(107, 107, 107)')),
                title='Feature Importance of Bottom 20 Features',
                barmode='group',bargap=0.4,bargroupgap=0.1)
fig17 = Figure(data=data, layout=layout)
iplot(fig17, filename='feature-importance-worst-mean-impurity-bar')
#py.plot(fig17, filename='bar-chart-countries', sharing='public')

A lot of the countries are very poorly performing features. I wonder why..

I am still continuing to tweak and improve this model to get a better prediction.

Few things that I can try next:

1. Define new features which are better predictors (maybe wage amount)
2. Regularization
3. Add more data
4. Tune hyperparameters of Random Forests and Logistic Regression
5. Try grid search

The next steps I am also going to try to pick up more skills are:

1. Build a web app that outputs processing times based on information of applicant
2. Try converting tables to a SQL database
3. Try some web scraping

In [49]:
#output predictor
#partial dependency plot

In [50]:
#perm_data_15[perm_data_15['JOB_INFO_WORK_STATE']=='PENNSYLVANIA']['TIME_CATEGORIES'].std()

In [51]:
#perm_data_15[perm_data_15['COUNTRY_OF_CITIZENSHIP']=='UZBEKISTAN']['TIME_CATEGORIES'].std()